# Podstawy i ewolucja architektur neuronowych. Część 1

## Spis treści

1. [Wprowadzenie](#wprowadzenie)
2. [Historia rozwoju sieci neuronowych](#11-historia-rozwoju-sieci-neuronowych)
3. [Kluczowe problemy klasycznych architektur](#12-kluczowe-problemy-klasycznych-architektur)
4. [Przełomowe innowacje techniczne](#13-przełomowe-innowacje-techniczne)
5. [Dlaczego potrzebujemy nowych architektur](#14-dlaczego-potrzebujemy-nowych-architektur)
6. [Podsumowanie modułu](#podsumowanie-modułu-1)

---

## Wprowadzenie

Aby zrozumieć nowoczesne architektury sieci neuronowych, musimy najpierw poznać historię ich rozwoju i kluczowe problemy, które doprowadziły do powstania innowacyjnych rozwiązań. Ten moduł przedstawia fundament, na którym zbudowane są wszystkie współczesne architektury.

---

## 1.1 Historia rozwoju sieci neuronowych

### Perceptron (1958) - Początek ery

Frank Rosenblatt stworzył **perceptron** - pierwszą implementację sztucznego neuronu zdolnego do uczenia się. Perceptron to prosty model liniowy, który może klasyfikować dane liniowo separowalne.

**Kluczowa idea:** Neuron sumuje ważone wejścia i aktywuje się, gdy suma przekracza próg.

```python
import numpy as np
import matplotlib.pyplot as plt

# Prosty perceptron dla problemu AND
class Perceptron:
    def __init__(self, lr=0.1, n_iter=100):
        self.lr = lr
        self.n_iter = n_iter
    
    def fit(self, X, y):
        self.weights = np.random.randn(X.shape[1]) * 0.01
        self.bias = 0
        
        for _ in range(self.n_iter):
            for xi, yi in zip(X, y):
                pred = 1 if np.dot(xi, self.weights) + self.bias >= 0 else 0
                update = self.lr * (yi - pred)
                self.weights += update * xi
                self.bias += update

# Problem AND
X = np.array([[0,0], [0,1], [1,0], [1,1]])
y = np.array([0, 0, 0, 1])

p = Perceptron()
p.fit(X, y)
print("Perceptron nauczył się AND!")
```

**Ograniczenie:** Nie może nauczyć się funkcji XOR (nie jest liniowo separowalna).

---

### Zima AI i przełom lat 80.

W latach 1969-1986 rozwój sieci neuronowych prawie zamarł z powodu niemożności rozwiązania problemu XOR i braku algorytmów trenowania wielowarstwowych sieci.

**Przełom:** Odkrycie **wstecznej propagacji błędu** (backpropagation) przez Rumelharta, Hintona i Williamsa w 1986 roku umożliwiło trenowanie wielowarstwowych sieci.

---

### Wsteczna propagacja błędu

Backpropagation to algorytm, który efektywnie oblicza gradienty funkcji straty względem wag sieci, wykorzystując **regułę łańcuchową**.

```python
# Prosta 2-warstwowa sieć z backpropagation
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.W1 = np.random.randn(input_size, hidden_size) * 0.5
        self.W2 = np.random.randn(hidden_size, output_size) * 0.5
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
    
    def forward(self, X):
        self.hidden = self.sigmoid(np.dot(X, self.W1))
        self.output = self.sigmoid(np.dot(self.hidden, self.W2))
        return self.output
    
    def backward(self, X, y, lr=0.1):
        # Gradient na wyjściu
        output_error = self.output - y
        output_delta = output_error * self.output * (1 - self.output)
        
        # Propagacja do warstwy ukrytej (reguła łańcuchowa!)
        hidden_error = output_delta.dot(self.W2.T)
        hidden_delta = hidden_error * self.hidden * (1 - self.hidden)
        
        # Aktualizacja wag
        self.W2 -= lr * self.hidden.T.dot(output_delta)
        self.W1 -= lr * X.T.dot(hidden_delta)

# XOR - niemożliwe dla perceptronu, ale możliwe dla sieci wielowarstwowej!
X_xor = np.array([[0,0], [0,1], [1,0], [1,1]])
y_xor = np.array([[0], [1], [1], [0]])

nn = NeuralNetwork(2, 4, 1)
for _ in range(5000):
    nn.forward(X_xor)
    nn.backward(X_xor, y_xor)

print("Sieć nauczyła się XOR!")
```

---

### Głębokie sieci neuronowe (2006-2012)

Renesans sieci neuronowych dzięki:
- **Pre-training** (Hinton, 2006)
- **ReLU activation** (szybsza niż sigmoid)
- **Dropout** (regularyzacja)
- **GPU computing** (tysiące razy szybsze trenowanie)

---

### Era ImageNet i przełom 2012 roku

**AlexNet (2012)** - przełom w deep learning:
- Pierwsza głęboka CNN, która wygrała ILSVRC
- Błąd: 16.4% (vs 26% tradycyjne metody)
- Wykorzystanie GPU, ReLU, dropout

```python
# Wizualizacja postępu ImageNet
years = [2010, 2011, 2012, 2013, 2014, 2015, 2017]
errors = [28.2, 25.8, 16.4, 11.7, 7.3, 3.6, 2.25]
models = ['Traditional', 'Traditional', 'AlexNet', 'ZFNet',
          'GoogLeNet', 'ResNet', 'SENet']

plt.figure(figsize=(10, 6))
plt.plot(years, errors, 'o-', linewidth=3, markersize=10)
plt.axhline(y=5.1, color='r', linestyle='--', label='Human-level')
plt.xlabel('Rok')
plt.ylabel('Top-5 Error (%)')
plt.title('Rewolucja Deep Learning (ImageNet)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()
```

---

## 1.2 Kluczowe problemy klasycznych architektur

### Problem zanikającego/eksplodującego gradientu

**Zanikający gradient:** W głębokich sieciach gradienty stają się coraz mniejsze, co powoduje, że wczesne warstwy uczą się bardzo wolno.

**Eksplodujący gradient:** Gradienty rosną wykładniczo, powodując niestabilność.

```python
# Symulacja zanikającego gradientu
def gradient_flow(n_layers, activation='sigmoid'):
    gradient = 1.0
    gradients = [gradient]
    
    for _ in range(n_layers):
        weight = np.random.randn() * 0.5
        # Sigmoid ma pochodną max 0.25
        activation_deriv = np.random.uniform(0.1, 0.25)
        gradient *= weight * activation_deriv
        gradients.append(abs(gradient))
    
    return gradients

# Porównanie sigmoid vs ReLU
grads_sigmoid = gradient_flow(30, 'sigmoid')
plt.semilogy(grads_sigmoid, label='Sigmoid')
plt.xlabel('Warstwa (od wyjścia do wejścia)')
plt.ylabel('Wielkość gradientu (log)')
plt.title('Zanikający gradient w głębokich sieciach')
plt.legend()
plt.grid(True)
plt.show()

print("Gradient zanika eksponencjalnie!")
```

**Rozwiązania:**
- ReLU i warianty (LeakyReLU, ELU)
- Batch Normalization
- Skip Connections (ResNet)
- Lepsza inicjalizacja wag (Xavier, He)

---

### Overfitting w głębokich sieciach

**Overfitting:** Model "zapamiętuje" dane treningowe zamiast uczyć się ogólnych wzorców.

```python
# Demonstracja overfittingu
from sklearn.model_selection import train_test_split

# Dane z szumem
X = np.linspace(0, 10, 100).reshape(-1, 1)
y = np.sin(X) + np.random.normal(0, 0.3, X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Duża sieć bez regularyzacji → overfit
# vs sieć z dropout/L2 → dobra generalizacja

# Wykres learning curves pokazuje różnicę między train a test loss
```

**Techniki zapobiegania:**
- **Dropout** - losowe wyłączanie neuronów
- **L1/L2 Regularization** - karanie dużych wag
- **Early stopping** - zatrzymanie gdy test loss przestaje maleć
- **Data augmentation** - zwiększanie zbioru treningowego
- **Batch Normalization** - efekt regularyzacyjny

---

### Ograniczenia w modelowaniu długoterminowych zależności

Klasyczne RNN miały problemy z "zapamiętywaniem" informacji z odległej przeszłości w sekwencjach.

```python
# Problem vanilla RNN: informacja zanika po kilkunastu krokach
class SimpleRNN:
    def forward_sequence(self, sequence):
        hidden = np.zeros(50)
        for x in sequence:
            hidden = np.tanh(np.dot(W_hh, hidden) + np.dot(W_xh, x))
        return hidden

# Po 20-30 krokach czasowych, pierwotna informacja praktycznie znika
```

**Rozwiązania:**
- LSTM (Long Short-Term Memory)
- GRU (Gated Recurrent Unit)
- **Attention Mechanism** - bezpośredni dostęp do wcześniejszych stanów
- **Transformers** - self-attention zamiast rekurencji

---

### Koszty obliczeniowe

Trenowanie dużych modeli wymaga ogromnych zasobów.

```python
# Porównanie rozmiarów modeli
models = ['AlexNet\n2012', 'GPT-2\n2019', 'GPT-3\n2020', 'GPT-4\n2023']
params = [60, 1500, 175000, 1760000]  # Miliony parametrów
costs = [10, 43, 4600, 100000]  # Tysiące USD

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.bar(models, params, color='skyblue')
plt.yscale('log')
plt.ylabel('Parametry (miliony)')
plt.title('Wzrost rozmiaru modeli')

plt.subplot(1, 2, 2)
plt.bar(models, costs, color='lightcoral')
plt.yscale('log')
plt.ylabel('Koszt treningu (tyś. USD)')
plt.title('Wzrost kosztów')
plt.tight_layout()
plt.show()
```

**Implikacje:**
- Bariery wejścia dla małych graczy
- Koszty środowiskowe (CO2)
- Potrzeba efektywniejszych architektur

---

## 1.3 Przełomowe innowacje techniczne

### Dropout

**Dropout** (Hinton, 2012) - losowe wyłączanie neuronów podczas treningu.

```python
# Dropout w praktyce
class DropoutLayer:
    def __init__(self, dropout_rate=0.5):
        self.rate = dropout_rate
    
    def forward(self, x, training=True):
        if training:
            mask = np.random.binomial(1, 1-self.rate, x.shape)
            return x * mask / (1 - self.rate)  # Scaling
        return x  # Bez dropout w inference
```

**Dlaczego działa:**
- Ensemble effect - trenujemy wiele "podsieci"
- Zmusza do robustności
- Redukuje ko-adaptację neuronów

---

### Batch Normalization

**Batch Normalization** (2015) - normalizuje aktywacje w każdej warstwie.

```python
# Batch Normalization
class BatchNorm:
    def forward(self, x, training=True):
        if training:
            mean = np.mean(x, axis=0)
            var = np.var(x, axis=0)
            x_norm = (x - mean) / np.sqrt(var + 1e-5)
        else:
            x_norm = (x - self.running_mean) / np.sqrt(self.running_var + 1e-5)
        
        # Scale and shift (learnable)
        return self.gamma * x_norm + self.beta
```

**Zalety:**
- Przyspiesza trening (wyższy learning rate)
- Stabilizuje trening
- Efekt regularyzacyjny

---

### Skip Connections i Residual Learning

**Skip Connections** (ResNet, 2015) - bezpośrednie połączenia między warstwami.

```python
# Residual block
class ResidualBlock:
    def forward(self, x):
        identity = x  # Skip connection
        
        # F(x) - funkcja uczona przez warstwy
        out = conv1(x)
        out = relu(out)
        out = conv2(out)
        
        # Dodanie skip connection
        out = out + identity  # y = F(x) + x
        out = relu(out)
        return out
```

**Dlaczego działa:**
- Gradient flow przez skip connections
- Łatwo nauczyć się identity mapping (F=0)
- Umożliwił sieci 100+ warstw

**Przełom:** ResNet wygrał ImageNet 2015 z błędem 3.6%

---

### Attention Mechanism

**Attention** (2014) - model sam decyduje, na które części wejścia patrzeć.

```python
# Uproszczony attention
def attention(query, keys, values):
    # 1. Oblicz similarity (Query · Key)
    scores = np.dot(query, keys.T) / np.sqrt(keys.shape[-1])
    
    # 2. Softmax → rozkład prawdopodobieństwa
    weights = softmax(scores)
    
    # 3. Ważona suma Values
    output = np.dot(weights, values)
    return output, weights
```

**Zalety:**
- Długie zależności bez problemu
- Przetwarzanie równoległe (vs RNN)
- Interpretowalność (widzimy wagi uwagi)
- Fundament Transformerów (GPT, BERT, etc.)

---

### Transfer Learning

**Transfer Learning** - wykorzystanie wiedzy z jednego problemu do rozwiązania innego.

```python
# Transfer learning w praktyce
# 1. Załaduj pre-trained model (np. ResNet na ImageNet)
model = load_pretrained_model('resnet50')

# 2. Zamroź wczesne warstwy
for layer in model.layers[:-5]:
    layer.trainable = False

# 3. Dodaj nowy classifier dla twojego zadania
model.add(Dense(num_classes, activation='softmax'))

# 4. Fine-tune na twoich danych
model.fit(your_data, your_labels)
```

**Zalety:**
- Mniej danych potrzebnych
- Szybszy trening
- Lepsza wydajność
- Demokratyzacja AI

**Strategie:**
- Feature extraction (frozen weights)
- Fine-tuning (aktualizacja wszystkich/niektórych warstw)
- Progressive fine-tuning

---

## 1.4 Dlaczego potrzebujemy nowych architektur

### Nowe dziedziny zastosowań

**Medycyna:**
- Diagnostyka obrazowa (CT, MRI)
- Odkrywanie leków
- Wymaga: interpretability, uncertainty estimation

**Autonomous Vehicles:**
- Percepcja otoczenia w czasie rzeczywistym
- Wymaga: real-time, high reliability

**Edge & Mobile:**
- Smartfony, IoT, wearables
- Wymaga: extreme efficiency, low latency

**Multimodal AI:**
- Vision + Language + Audio
- Wymaga: unified architectures

---

### Rosnące wymagania

```python
# Trendy w wymaganiach (2015 = baseline)
years = np.arange(2015, 2026)
plt.figure(figsize=(10, 6))
plt.plot(years, np.exp((years-2015)*0.3), label='Rozmiar danych')
plt.plot(years, np.exp((years-2015)*0.25), label='Złożoność zadań')
plt.plot(years, np.exp((years-2015)*0.2), label='Real-time requirements')
plt.xlabel('Rok')
plt.ylabel('Wzrost względem 2015')
plt.title('Rosnące wymagania dla architektur AI')
plt.legend()
plt.yscale('log')
plt.grid(True, alpha=0.3)
plt.show()
```

---

### Kierunki rozwoju

**Efficiency & Scale:**
- Sparse models (Mixture of Experts)
- Quantization, pruning
- Neural Architecture Search

**Better Learning:**
- Few-shot, zero-shot learning
- Self-supervised learning
- Continual learning

**New Paradigms:**
- State Space Models (Mamba, S4)
- Graph Neural Networks
- Geometric Deep Learning

**Interpretability & Trust:**
- Explainable AI (XAI)
- Uncertainty quantification
- Fairness & robustness

---

## Podsumowanie Modułu 1

### Kluczowe punkty historyczne:
- **Perceptron (1958)** - początek, ograniczony do problemów liniowych
- **Backpropagation (1986)** - umożliwił wielowarstwowe sieci
- **AlexNet (2012)** - przełom deep learning
- **ResNet (2015)** - skip connections, sieci 100+ warstw

### Główne problemy i rozwiązania:
| Problem | Rozwiązanie |
|---------|-------------|
| Zanikający gradient | ReLU, Batch Norm, Skip Connections |
| Overfitting | Dropout, regularyzacja, augmentacja |
| Długie zależności | LSTM, GRU, Attention, Transformers |
| Koszty obliczeniowe | Efektywne architektury, transfer learning |

### Przełomowe innowacje:
- ✅ **Dropout** - regularyzacja przez losowe wyłączanie
- ✅ **Batch Normalization** - stabilizacja treningu
- ✅ **Skip Connections** - głębokie sieci (ResNet)
- ✅ **Attention** - fundament Transformerów
- ✅ **Transfer Learning** - demokratyzacja AI

### Współczesne wyzwania:
- Efektywność (edge computing)
- Multimodalność
- Długie konteksty
- Interpretability
- Zrównoważony rozwój (Green AI)

---

## Co dalej?

W **Modułu 2** zagłębimy się w konkretne rodziny architektur:
- Sieci konwolucyjne (CNN → ResNet → EfficientNet → ConvNeXt)
- Transformery i mechanizmy uwagi (ViT, BERT, GPT)
- Architektury rekurencyjne i ich następcy
- Podejścia hybrydowe

Każda z tych rodzin rozwiązuje inne problemy i ma swoje zastosowania!

---

**Pytania do przemyślenia:**
1. Które z przedstawionych innowacji uważasz za najbardziej przełomowe?
2. Jakie nowe problemy mogą wymagać kolejnych przełomów?
3. Dokąd zmierza rozwój architektur w następnej dekadzie?